# Notes
- utterances have repetition removed, only utterances above 3 words are considered. removed utterances which were only erm, mmm mm, mm, er etc
- utterances are merged in groups of threes in conversation (so in each example, there are three utterances, possibly from different speakers)

In [394]:
import pandas as pd
import os
import re
import pathlib
from bs4 import BeautifulSoup, Tag
import string

In [395]:
data_dir = str(pathlib.Path().resolve().parent) + "\\datafiles\\spoken corpus\\bnc\\spoken\\untagged\\"
save_dir = str(pathlib.Path().resolve().parent) + "\\datafiles\\spoken corpus\\bnc\\preprocessed_bnc.csv"

In [396]:
all_utterances = []

In [397]:
import nltk

def tuple_to_str(tuple):
    output_str = ""
    for tup in tuple:
        try:
            output_str += " " + tup[0]
            output_str += " " + tup[1]
        except:
            continue
    return output_str[1:]


def remove_single_repetitions(text):
    """
    Removes duplicated words (stuttering) and duplicated pauses from utterance.
    e.g: I I I I I I wanted --> I wanted.
    :param line: input text containing dupes.
    :return: unduped string containing text.
    """
    utterance = text.split(" ")

    newlist = []
    newlist.append(utterance[0])
    for i, element in enumerate(utterance):
        if i > 0 and utterance[i - 1] != element:
            newlist.append(element)

    return ' '.join(newlist)


def remove_bigram_repetitions(text):
    """
    Removes bigram stuttering from text. I went I went to the to the doctor --> I went to the doctor.
    :param text: input text containing a string
    :return: string without duplicates.
    """
    bigram = list(nltk.bigrams(text.split()))
    grams = []

    for i in range(0, len(bigram)):
        if i % 2 == 0:
            grams.append(bigram[i])

    result = []
    prev_item = None
    for item in grams:
        if item != prev_item:
            result.append(item)
            prev_item = item

    if result[-1][-1] != bigram[-1][-1]:
        result.append(tuple((bigram[-1][-1]).split(" ")))

    return tuple_to_str(result)

def remove_all_repetitions(text):
    """
    Removes bigram repetitions and stuttering from text.
    :return: Clean text.
    """
    try:
        output_text = remove_single_repetitions(text)
        output_text2 = remove_bigram_repetitions(output_text)
    except:
        return text
    return output_text2

In [398]:
for subdir, dirs, files in os.walk(data_dir):
    for file in files:
        dirname = os.path.basename(subdir)
        if str(data_dir + dirname + file).endswith(".xml"):
            with open(data_dir + dirname + file, 'r', encoding="utf8") as f:
                data = f.read()
                soup = BeautifulSoup(data, 'xml')
                three_utterances = []
                utterances = soup.findChildren('u')#, {"n": True})
                print(file)
                for utt in utterances:
                    utterance = ""
                    for elem in utt:
                        if isinstance(elem, Tag):
                            # if pause add eos
                            if elem.name == 'pause' and elem.attrs.get('dur'):
                                utterance = utterance.rstrip() + '. '
                        if elem.get_text().strip():                               
                            if isinstance(elem, Tag): 
                                if elem.name == "unclear":
                                    utterance = utterance.strip() + " " + elem.get_text().strip() + " "
                            else:
                                utterance += elem.get_text().strip() + " "
                    # print(utterance)
                    if len(re.findall("[a-zA-Z_]+", utterance)) >= 3:
                        if utterance.rstrip().lstrip()[-1] not in ['!', '?']:
                            utterance = utterance.rstrip().lstrip()+'.'
                        else:
                            utterance = utterance.rstrip().lstrip()
                        utterance = re.split('(?<=[\.\?\!])\s*', utterance)
                        new_list = []
                        for u in utterance:
                            new_u = u.translate(str.maketrans('','',string.punctuation)).replace(' ', '')
                            if list(set(new_u))!=['e', 'r', 'm'] and list(set(new_u))!=['m'] and list(set(new_u))!=['e','r']:
                                if u and len(re.findall("[a-zA-Z_]+", u)) >= 3:
                                    new_list.append(remove_all_repetitions(u))   
                            # if new_u:
                            #     print(u, list(set(new_u)), list(set(new_u))!=['e', 'r', 'm'] and list(set(new_u))!=['m'] and list(set(new_u))!=['e','r'])
                        three_utterances.extend(new_list)
                        # print(three_utterances)
                        # print("----------------------------")
                    if len(three_utterances) == 3:
                        all_utterances.append(' '.join(three_utterances))
                        three_utterances = []
                    if len(three_utterances) > 3:
                        all_utterances.append(' '.join(three_utterances[0:3]))
                        three_utterances = three_utterances[3:]
            # break

S23A.xml
S24A.xml
S24D.xml
S24E.xml
S263.xml
S26N.xml
S27D.xml
S28F.xml
S29Q.xml
S29X.xml
S2A5.xml
S2AJ.xml
S2AX.xml
S2B5.xml
S2C9.xml
S2CY.xml
S2DD.xml
S2E2.xml
S2EF.xml
S2FQ.xml
S2FT.xml
S2GC.xml
S2GS.xml
S2JK.xml
S2JV.xml
S2K6.xml
S2K7.xml
S2KP.xml
S2LC.xml
S2LD.xml
S2NQ.xml
S2PS.xml
S2PY.xml
S2QU.xml
S2RD.xml
S2T6.xml
S2TC.xml
S2TP.xml
S2U9.xml
S2UJ.xml
S2UT.xml
S2W4.xml
S2XJ.xml
S2XV.xml
S2Y5.xml
S2YK.xml
S2ZU.xml
S323.xml
S32W.xml
S32Y.xml
S33B.xml
S33N.xml
S346.xml
S355.xml
S35K.xml
S35U.xml
S36Z.xml
S376.xml
S37E.xml
S37F.xml
S37K.xml
S37Y.xml
S382.xml
S38F.xml
S38P.xml
S38V.xml
S39D.xml
S3A9.xml
S3AC.xml
S3AV.xml
S3C6.xml
S3CK.xml
S3CP.xml
S3DA.xml
S3GS.xml
S3JF.xml
S3KY.xml
S3LE.xml
S3M4.xml
S3M9.xml
S3MW.xml
S3N4.xml
S3QD.xml
S3RL.xml
S3RN.xml
S3RZ.xml
S3S6.xml
S3SA.xml
S3SU.xml
S3TD.xml
S3TZ.xml
S3U3.xml
S3UC.xml
S3VM.xml
S3XC.xml
S3YZ.xml
S432.xml
S43M.xml
S43S.xml
S452.xml
S457.xml
S45F.xml
S46J.xml
S46Q.xml
S477.xml
S47C.xml
S48K.xml
S49H.xml
S49V.xml
S4BA.xml
S4C2.xml
S

In [399]:
all_utterances

["it's a games word? like a computer games word? oh that's nice.",
 "I it's something I have really heard z-buffer fighting on a shimmering object. the old z-buffering. oh I know the one yeah.",
 "thank you for cooking. thank you very much. see what it's like.",
 "I haven't tasted what it's like. I must be half an hour in to cook. rice is good.",
 "I imagined it being making the sprouts nicer. it's a mighty green. I don't think it's overly orange.",
 "ah I need green. been missing out on the greens a little bit. I've been having yeah.",
 "misses the greens. I have been missing them. I usually have greens every day and haven't had so many of them recently.",
 "really miss them. see interesting that you sent me the text going but fruit and veg I though mm she sounds like she needs greens. that wasn't even me that said that was.",
 "but Co-op has loads of like reduced vegetables. but it's like. yeah I help yourself to all these different types of pickles on the side.",
 "have you got wate

In [400]:
df = pd.DataFrame(columns=["preprocessed_text"])
preprocessed = [str(x).strip() for x in all_utterances]
preprocessed  = [x for x in preprocessed if x != ""]
preprocessed = [x for x in preprocessed if len(re.findall("[a-zA-Z_]+", x)) >= 3]

In [401]:
df["preprocessed_text"] = preprocessed

In [402]:
df

,preprocessed_text
0,it's a games word? like a computer games word?...
1,I it's something I have really heard z-buffer ...
2,thank you for cooking. thank you very much. se...
3,I haven't tasted what it's like. I must be hal...
4,I imagined it being making the sprouts nicer. ...
...,...
330528,Sha Li 's quite nice. Sha Li yeah er but then ...
330529,I think he had other. another agenda as well. ...
330530,to like Mai Li which is like beautiful. I was ...
330531,I'll just stick with it yeah. this was like ha...


In [403]:
# extra preprocessing done before modifying the sentences through linguistic approach
df.to_csv(save_dir)